Link to Kaggle dataset: https://www.kaggle.com/datasets/deathtrooper/multichannel-glaucoma-benchmark-dataset/data

In [1]:
import zipfile
import pandas as pd
import keras
import os
import numpy as np
from keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D, Conv2D, BatchNormalization, MaxPooling2D, Input, Concatenate, ReLU, AveragePooling2D, UpSampling2D, GlobalMaxPooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adam
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v3 import preprocess_input 
from keras.layers import GlobalMaxPool2D

In [2]:
# Unzip files for processing
with zipfile.ZipFile("full-fundus.zip","r") as zip_ref:
    zip_ref.extractall("full-fundus")


In [3]:
# Read in metadata, and filter for a specific subset of images
# TODO: will add more variety of images later
image_data = pd.read_csv('metadata.csv')
image_data = image_data[image_data['names'].str.contains('FIVES', case=True) | image_data['names'].str.contains('HAGIS', case=True)
                       | image_data['names'].str.contains('LES-AV', case=True) | image_data['names'].str.contains('G1020', case=True)
                       | image_data['names'].str.contains('OIA', case=True) | image_data['names'].str.contains('ORIGA', case=True)]
#g1020 = image_data[image_data['names'].str.contains('G1020', case=True)]
#g1020 = g1020[g1020["fundus_oc_seg"] != "Not Visible"]

#image_data = pd.concat([subset_image_data, g1020])
image_data = image_data[['types', 'fundus', 'names']]

In [4]:
# There are some invalid data types for relevant columns
print(image_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 6596 entries, 0 to 12448
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   types   6596 non-null   int64 
 1   fundus  6596 non-null   object
 2   names   6596 non-null   object
dtypes: int64(1), object(2)
memory usage: 206.1+ KB
None


In [5]:
# Change datatypes to the desired values
image_data['types'] = image_data['types'].astype('string')
image_data['fundus'] = image_data['fundus'].astype('string')
image_data['names'] = image_data['names'].astype('string')
image_data['image_names'] = image_data['names'] + '.png'


In [6]:
print(image_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 6596 entries, 0 to 12448
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   types        6596 non-null   string
 1   fundus       6596 non-null   string
 2   names        6596 non-null   string
 3   image_names  6596 non-null   string
dtypes: string(4)
memory usage: 257.7 KB
None


In [7]:
# Randomize data 
image_data_random = image_data.sample(frac=1, random_state=1)

# Split into healthy and glaucoma positive sets
healthy = image_data_random[image_data_random['types'] == "0"]
glaucoma = image_data_random[image_data_random['types'] == "1"]
inconclusive = image_data_random[image_data_random['types'] == "-1"]

print(f"Number of healthy samples: {len(healthy)}")
print(f"Number of unhealthy samples: {len(glaucoma)}")
print(f"Number of inconclusive samples: {len(inconclusive)}")

Number of healthy samples: 5618
Number of unhealthy samples: 926
Number of unhealthy samples: 52


In [8]:
# Partition data into test and train sets
healthy_train_size = 3932
glaucoma_train_size = 530
inconclusive_train_size = 36

healthy_test_subset = image_data_random.head(len(healthy) - healthy_train_size)
glaucoma_test_subset = image_data_random.head(len(glaucoma) - glaucoma_train_size)
inconclusive_test_subset = image_data_random.head(len(inconclusive) - inconclusive_train_size)
test_subset = pd.concat([healthy_test_subset, glaucoma_test_subset, inconclusive_test_subset])

healthy_train_subset = image_data_random.tail(healthy_train_size)
glaucoma_train_subset = image_data_random.tail(glaucoma_train_size)
inconclusive_train_subset = image_data_random.tail(inconclusive_train_size)
train_subset = pd.concat([healthy_train_subset, glaucoma_train_subset, inconclusive_train_subset])

print(f"Test size: {len(test_subset)}")
print(f"Train size: {len(train_subset)}")

Test size: 2098
Train size: 4498


In [9]:
train_data_generator = ImageDataGenerator(preprocessing_function= preprocess_input)

flow_train_data = train_data_generator.flow_from_dataframe(dataframe=train_subset, 
                                            batch_size= 8, 
                                            shuffle=True, 
                                            x_col="image_names", 
                                            y_col="types", 
                                            validate_filenames=True, 
                                            target_size=(224, 224), 
                                            directory='full-fundus/full-fundus/', 
                                            color_mode='rgb')

test_data_generator = ImageDataGenerator(preprocessing_function= preprocess_input)

flow_test_data = train_data_generator.flow_from_dataframe(dataframe=test_subset, 
                                            batch_size= 1, 
                                            shuffle=False, 
                                            x_col="image_names", 
                                            y_col="types", 
                                            validate_filenames=True, 
                                            target_size=(224, 224), 
                                            directory='full-fundus/full-fundus/', 
                                            color_mode='rgb')



Found 4498 validated image filenames belonging to 3 classes.
Found 2098 validated image filenames belonging to 3 classes.


In [10]:
def conv_model(image_size):
    
    m_input = keras.Input(shape=(image_size, image_size, 3))
    transfer = keras.applications.MobileNetV3Large(
        weights='imagenet', include_top= False, input_tensor= m_input, alpha=0.75
    )
    m_output = Dropout(0.5)(transfer.output)
    m_output = Conv2D(filters=256, kernel_size=1)(m_output) 
    m_output = GlobalMaxPool2D()(m_output)
    m_output = Dropout(0.5)(m_output)
    m_output = Dense(3, activation='softmax')(m_output)

    return keras.Model(inputs=m_input, outputs=m_output)

model = conv_model(image_size=224)

# Reduce the learning rate if an epoch occurs where there is no improvement to the output of the loss function
reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.8, patience=1, min_lr=1e-6)

# Using the Adam optimizer with binary cross entropy, compile the model using the given metrics
model.compile(optimizer= Adam(1e-6), 
              loss='hinge', 
              metrics=['accuracy'])

trained_model = model.fit(flow_train_data, 
                    steps_per_epoch= len(flow_train_data) // 4,
                    validation_data= flow_test_data, 
                    validation_steps= len(flow_test_data), 
                    epochs=4, 
                    callbacks=[reduce_lr])

Epoch 1/4


/Users/ThomasMcBride/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


140/140 ━━━━━━━━━━━━━━━━━━━━ 94s 601ms/step - accuracy: 0.2496 - loss: 1.1617 - val_accuracy: 0.0362 - val_loss: 1.2950 - learning_rate: 1.0000e-06
Epoch 2/4
140/140 ━━━━━━━━━━━━━━━━━━━━ 49s 350ms/step - accuracy: 0.3305 - loss: 1.1128 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - learning_rate: 1.0000e-06
Epoch 3/4


2024-03-30 12:20:54.905506: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/ThomasMcBride/anaconda3/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


140/140 ━━━━━━━━━━━━━━━━━━━━ 107s 768ms/step - accuracy: 0.3209 - loss: 1.1224 - val_accuracy: 0.0715 - val_loss: 1.2722 - learning_rate: 1.0000e-06
Epoch 4/4
140/140 ━━━━━━━━━━━━━━━━━━━━ 43s 310ms/step - accuracy: 0.3220 - loss: 1.1203 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - learning_rate: 1.0000e-06


2024-03-30 12:23:25.586009: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [11]:
# Demonstration:
img_path = 'full-fundus/full-fundus/PAPILA-1.png'
img = image.load_img(img_path, target_size=(224, 224))
 
img_array_representation = np.expand_dims(image.img_to_array(img), axis=0)
img_array_representation = preprocess_input(img_array_representation)

prediction = model.predict(img_array_representation)
print(prediction)
# Interpret the prediction
if prediction[0][0] > prediction[0][1] and prediction[0][0] > prediction[0][2]:
    print(f"Prediction: Glaucoma, with {prediction[0][0] * 100}% confidence")
elif prediction[0][1] > prediction[0][0] and prediction[0][1] > prediction[0][2]:
    print(f"Prediction: Healthy, with {prediction[0][1] * 100}% confidence")
else: 
    print(f"Prediction: Glaucoma Suspected, with {prediction[0][2] * 100}% confidence")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 663ms/step
[[9.9612159e-01 3.1514929e-07 3.8780558e-03]]
Prediction: Glaucoma, with 99.61215853691101% confidence


In [12]:
img_path = 'full-fundus/full-fundus/PAPILA-2.png'
img = image.load_img(img_path, target_size=(224, 224))
 
img_array_representation = np.expand_dims(image.img_to_array(img), axis=0)
img_array_representation = preprocess_input(img_array_representation)  
prediction = model.predict(img_array_representation)
print(prediction)
# Interpret the prediction
if prediction[0][0] > prediction[0][1] and prediction[0][0] > prediction[0][2]:
    print(f"Prediction: Glaucoma, with {prediction[0][0] * 100}% confidence")
elif prediction[0][1] > prediction[0][0] and prediction[0][1] > prediction[0][2]:
    print(f"Prediction: Healthy, with {prediction[0][1] * 100}% confidence")
else: 
    print(f"Prediction: Glaucoma Suspected, with {prediction[0][2] * 100}% confidence")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[9.8653328e-01 1.4083834e-04 1.3325771e-02]]
Prediction: Glaucoma, with 98.65332841873169% confidence


In [13]:
img_path = 'full-fundus/full-fundus/PAPILA-21.png'
img = image.load_img(img_path, target_size=(224, 224))
 
img_array_representation = np.expand_dims(image.img_to_array(img), axis=0)
img_array_representation = preprocess_input(img_array_representation) 

prediction = model.predict(img_array_representation)
print(prediction)
# Interpret the prediction
if prediction[0][0] > prediction[0][1] and prediction[0][0] > prediction[0][2]:
    print(f"Prediction: Glaucoma, with {prediction[0][0] * 100}% confidence")
elif prediction[0][1] > prediction[0][0] and prediction[0][1] > prediction[0][2]:
    print(f"Prediction: Healthy, with {prediction[0][1] * 100}% confidence")
else: 
    print(f"Prediction: Glaucoma Suspected, with {prediction[0][2] * 100}% confidence")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[0.10825618 0.09698095 0.7947629 ]]
Prediction: Glaucoma Suspected, with 79.4762909412384% confidence


In [14]:
img_path = 'full-fundus/full-fundus/PAPILA-22.png'
img = image.load_img(img_path, target_size=(224, 224))
 
img_array_representation = np.expand_dims(image.img_to_array(img), axis=0)
img_array_representation = preprocess_input(img_array_representation) 

prediction = model.predict(img_array_representation)
print(prediction)
# Interpret the prediction
if prediction[0][0] > prediction[0][1] and prediction[0][0] > prediction[0][2]:
    print(f"Prediction: Glaucoma, with {prediction[0][0] * 100}% confidence")
elif prediction[0][1] > prediction[0][0] and prediction[0][1] > prediction[0][2]:
    print(f"Prediction: Healthy, with {prediction[0][1] * 100}% confidence")
else: 
    print(f"Prediction: Glaucoma Suspected, with {prediction[0][2] * 100}% confidence")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[[0.77369654 0.19523463 0.03106887]]
Prediction: Glaucoma, with 77.36965417861938% confidence


In [15]:
img_path = 'full-fundus/full-fundus/PAPILA-11.png'
img = image.load_img(img_path, target_size=(224, 224))
 
img_array_representation = np.expand_dims(image.img_to_array(img), axis=0)
img_array_representation = preprocess_input(img_array_representation) 
prediction = model.predict(img_array_representation)
print(prediction)
# Interpret the prediction
if prediction[0][0] > prediction[0][1] and prediction[0][0] > prediction[0][2]:
    print(f"Prediction: Glaucoma, with {prediction[0][0] * 100}% confidence")
elif prediction[0][1] > prediction[0][0] and prediction[0][1] > prediction[0][2]:
    print(f"Prediction: Healthy, with {prediction[0][1] * 100}% confidence")
else: 
    print(f"Prediction: Glaucoma Suspected, with {prediction[0][2] * 100}% confidence")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[0.02374464 0.10970604 0.8665493 ]]
Prediction: Glaucoma Suspected, with 86.65493130683899% confidence
